In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd gdrive/My Drive/dvl/Mask_RCNN

/content/gdrive/My Drive/dvl/Mask_RCNN


In [ ]:
# check versions of tf and keras
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)
#!pip install keras==2.1.6
import keras
keras.__version__

TensorFlow 1.x selected.
1.15.2


Using TensorFlow backend.


'2.3.1'

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log
from mrcnn.config import Config

from pycocotools.coco import COCO
from pycocotools import mask as maskUtils

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")




```
# Als Code formatiert
```

#**Configurations**

In [ ]:
############################################################
#  Configurations
############################################################

class BasilConfig(Config):
    """Configuration for training on the nucleus segmentation dataset."""
    # Give the configuration a recognizable name
    NAME = "basil"

    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + leaf

    # Number of training and validation steps per epoch
    STEPS_PER_EPOCH = 75 
    VALIDATION_STEPS = 26 

    # Don't exclude based on confidence. Since we have two classes
    # then 0.5 is the minimum anyway as it picks between nucleus and BG
    DETECTION_MIN_CONFIDENCE = 0.5

    # Backbone network architecture
    # Supported values are: resnet50, resnet101
    BACKBONE = "resnet101" ## change for different models

    # Select one entity (e.g. bounding boxes) out of many overlapping entities
    DETECTION_NMS_THRESHOLD = 0.3       

    # Length of square anchor side in pixels
    RPN_ANCHOR_SCALES = (16, 32, 64, 128, 256)

    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000

    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.9

    # How many anchors per image to use for RPN training
    RPN_TRAIN_ANCHORS_PER_IMAGE = 300  ## change for models (range 200 - 300)

    # Image mean (RGB)
    #MEAN_PIXEL = np.array([43.53, 39.56, 48.22])

    # If enabled, resizes instance masks to a smaller size to reduce
    # memory load. Recommended when using high-resolution images.
    USE_MINI_MASK = True
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask

    # Number of ROIs per image to feed to classifier/mask heads
    TRAIN_ROIS_PER_IMAGE = 300  ##  change for models (range 200 - 300) 

    # Maximum number of ground truth instances to use in one image
    MAX_GT_INSTANCES = 40  ##  change for models (30) 

    # Max number of final detections per image
    DETECTION_MAX_INSTANCES = 60  ##  change for models (range 200 - 300) 


class BasilInferenceConfig(BasilConfig):
    # Set batch size to 1 to run one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    # Don't resize imager for inferencing
   # IMAGE_RESIZE_MODE = "pad64"
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.7

### CURRENT MODEL ####
current_model = 'model_9'

# Path to a specific weights file
weights_path = os.path.join(ROOT_DIR, "Mask_RCNN/DEFAULT_LOGS_DIR/Model_9/mask_rcnn_basil_0015.h5")

In [ ]:
############################################################
#  Dataset
############################################################

class BasilDataset(utils.Dataset):

    def load_basil(self, dataset_dir, subset):
        """Load a subset of the COCO dataset.
        dataset_dir: The root directory of the COCO dataset.
        subset: What to load (train, val, test)
        class_ids: If provided, only loads images that have the given classes.
        """

        # load json with annotated images in COCO format
        coco = COCO("../segments2/kailkuhn_playground/v08.json")

        all_image_ids = list(coco.imgs.keys())

        # handle sets of train and val
        image_ids = []

        if subset == "train": #1-151
                for image in all_image_ids:
                    if image <= len(all_image_ids)*.70:
                        image_ids.append(image)

        elif subset == "val": #152-205
            for image in all_image_ids:
                if image > len(all_image_ids)*.70 and image <= len(all_image_ids)*.95:
                    image_ids.append(image)

        elif subset == "test": #206-216
            for image in all_image_ids:
                if image > len(all_image_ids)*.95:
                    image_ids.append(image)

        
        # Load all classes or a subset?
        class_ids = sorted(coco.getCatIds())

        # Add the class
        self.add_class("basil", 1, 'leaf')

        # Add images
        for i in image_ids:
           # print(i)
            self.add_image(
                "basil", image_id=i,
                path=os.path.join("../segments2/kailkuhn_playground/v.08", coco.imgs[i]['file_name']),
                width=coco.imgs[i]["width"],
                height=coco.imgs[i]["height"],
                annotations=coco.loadAnns(coco.getAnnIds(
                    imgIds=[i], catIds=class_ids, iscrowd=None)))



    def load_mask(self, image_id):
        """Load instance masks for the given image.

        Different datasets use different ways to store masks. This
        function converts the different mask format to one format
        in the form of a bitmap [height, width, instances].

        Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a COCO image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "basil":
            print('wrong source')

        instance_masks = []
        class_ids = []
  #      print(self.image_info[image_id])
  #      print(self.image_info[image_id]["annotations"])
        annotations = self.image_info[image_id]["annotations"]
        # Build mask of shape [height, width, instance_count] and list
        # of class IDs that correspond to each channel of the mask.
        for annotation in annotations:
           # print( 'category if line 71' + str(annotation['category_id']))  # just hardcode 0?
            class_id = self.map_source_class_id(
                "basil.{}".format(annotation['category_id'])) 
            if class_id:
                m = self.annToMask(annotation, image_info["height"],
                                   image_info["width"])
                # Some objects are so small that they're less than 1 pixel area
                # and end up rounded out. Skip those objects.
                if m.max() < 1:
                    continue
                # Is it a crowd? If so, use a negative class ID.
                if annotation['iscrowd']:
                    # Use negative class ID for crowds
                    class_id *= -1
                    # For crowd masks, annToMask() sometimes returns a mask
                    # smaller than the given dimensions. If so, resize it.
                    if m.shape[0] != image_info["height"] or m.shape[1] != image_info["width"]:
                        m = np.ones([image_info["height"], image_info["width"]], dtype=bool)
                instance_masks.append(m)
                class_ids.append(class_id)

        # Pack instance masks into an array
        if class_ids:
            mask = np.stack(instance_masks, axis=2)
            class_ids = np.array(class_ids, dtype=np.int32)
            return mask, class_ids
        else:
            # Call super class to return an empty mask
            print('calling return of parent')
            return super(BasilDataset, self).load_mask(image_id)


    # The following two functions are from pycocotools with a few changes.

    def annToRLE(self, ann, height, width):
        """
        Convert annotation which can be polygons, uncompressed RLE to RLE.
        :return: binary mask (numpy 2D array)
        """
        segm = ann['segmentation']
        if isinstance(segm, list):
            # polygon -- a single object might consist of multiple parts
            # we merge all parts into one mask rle code
            rles = maskUtils.frPyObjects(segm, height, width)
            rle = maskUtils.merge(rles)
        elif isinstance(segm['counts'], list):
            # uncompressed RLE
            rle = maskUtils.frPyObjects(segm, height, width)
        else:
            # rle
            rle = ann['segmentation']
        return rle

    def annToMask(self, ann, height, width):
        """
        Convert annotation which can be polygons, uncompressed RLE, or RLE to binary mask.
        :return: binary mask (numpy 2D array)
        """
        rle = self.annToRLE(ann, height, width)
        m = maskUtils.decode(rle)
        return m


#**Set Configurations**

In [ ]:
# Dataset directory
DATASET_DIR = os.path.join(ROOT_DIR, 'segments2/kailkuhn_playground/v.08')

# Inference Configuration
config = BasilInferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        60
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE          

# **Notebook Preferences**

In [ ]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    fig, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    fig.tight_layout()
    return ax

# **Load Validation Dataset**

In [ ]:
# Load validation dataset
dataset = BasilDataset()
dataset.load_basil(DATASET_DIR, "test")
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
Images: 11
Classes: ['BG', 'leaf']


# **Load Model**

In [ ]:
model = modellib.MaskRCNN(mode="inference",
                              model_dir= 'DEFAULT_LOGS_DIR',
                              config=config)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


In [ ]:


# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

Loading weights  /content/gdrive/My Drive/dvl/Mask_RCNN/DEFAULT_LOGS_DIR/Model_9/mask_rcnn_basil_0015.h5


In [ ]:
############################################################
#  RUN DETECTION and visualize
############################################################


######################
image_ids = dataset.image_ids

for image_id in image_ids:

    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset, config, image_id, use_mini_mask=True)


    info = dataset.image_info[image_id]

    print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, dataset.image_reference(image_id))) # image reference is not used


    # Run object detection
    results = model.detect([image], verbose=1)

    # Display results
    ax = get_ax(1)
    r = results[0]
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                dataset.class_names, r['scores'], ax=ax,
                                title="Predictions")

    plt.savefig(f'{image_id}_{current_model}.png')

    log("gt_class_id", gt_class_id)
    log("gt_bbox", gt_bbox)
    log("gt_mask", gt_mask)

In [ ]:
#######################
#### Compute IoU   ####
#######################

import statistics 
# # ##### results  Kail
image_ids = dataset.image_ids 


config.BATCH_SIZE = 1

def compute_batch_iou(dataset, image_ids, verbose=1):
    IoUs = []
    for image_id in image_ids:
        # Load image
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset, config,
                                   image_id, use_mini_mask=False)
        # Run object detection
        #results = model.detect_molded(image[np.newaxis], image_meta[np.newaxis], verbose=0)
        results = model.detect([image], verbose=0)
        # Compute Iou over range 0.5 to 0.95
        r = results[0]




        iou = utils.compute_overlaps_masks(gt_mask, r['masks'])
        IoUs.append(iou)
        
    return IoUs



def compute_final_iou(iou_list):
# compute_overlaps_masks() takes two arguments: mask1 and mask2. 
# Assume mask1 has m instances, and mask2 has n instances, 
# the function return an m*n array A. A[i, j] represents the IoU of ith instances of mask1 and jth instances of mask2.
    np.set_printoptions(suppress=True)

    res = []
    mydict = {}

    #iterate over true masks
    for i in range(len(iou_list)): # number of pictures
    # iterate over predicte masks
        res = []
        for j  in range(len(iou_list[i])): # true masks 16
            
            mask_max =iou_list[i][j].max() # maximum match in mask
            # all max matches of true masks
            res.append(mask_max)
            
            # save image ID (i) and result of IoU
        mydict[i] = res

    my_dictionary = {k: round(statistics.mean(v),3) for k, v in mydict.items()}
    
    return my_dictionary

def compute_leaf_error(IoUs):
    """ computes the percentage of change between true masks and
        predicted masks"""
    mydict = {}
    #iterate over true masks
    for i in range(len(IoUs)): # number of pictures
        error = round((IoUs[i].shape[0] - IoUs[i].shape[1]) / IoUs[i].shape[1], 3)
        mydict[i] = error

    return mydict



# run 
ious = compute_batch_iou(dataset, image_ids)
ious_perc = compute_final_iou(ious)
leaf_error = compute_leaf_error(ious)




In [ ]:
print("the leaf error per image are:", leaf_error)

count = 0
_sum = 0
for key in leaf_error:
    count += 1
    _sum += leaf_error[key]

print('this is the mean: ', _sum/count)

the leaf error per image are: {0: -0.059, 1: -0.133, 2: 0.083, 3: 0.125, 4: -0.067, 5: -0.071, 6: 0.0, 7: -0.105, 8: 0.214, 9: 0.25, 10: -0.222}
this is the mean:  0.0013636363636363648


In [ ]:
print("the ious per image are:", ious_perc)

count = 0
_sum = 0
for key in ious_perc:
    count += 1
    _sum += ious_perc[key]

print('this is the mean: ', _sum/count)

the ious per image are: {0: 0.744, 1: 0.818, 2: 0.52, 3: 0.775, 4: 0.845, 5: 0.565, 6: 0.754, 7: 0.616, 8: 0.652, 9: 0.591, 10: 0.781}
this is the mean:  0.6964545520869169


In [ ]:
import json

# save iou
with open(f'DEFAULT_LOGS_DIR/results_test/{current_model}_iou.json', 'w') as fp:
    json.dump(str(ious_perc), fp)

# save iou
with open(f'DEFAULT_LOGS_DIR/results_test/{current_model}_error.json', 'w') as fp:
    json.dump(str(leaf_error), fp)


# # load json
# with open('DEFAULT_LOGS_DIR/model_1.json', 'r') as fp:
#     data = json.load(fp)

# convert strin back to dict
# data1 = eval(data)

In [ ]:
current_model

'model_9'